In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import optuna
import os
import json
import warnings
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")

# === Load and prepare dataset ===
df = pd.read_csv("cleaned_stock_data_without_arm.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Company', 'Date'])

# Fix commas to save as numeric
df['Price Close'] = df['Price Close'].astype(str).str.replace(",", ".", regex=False)
df['Price Close'] = pd.to_numeric(df['Price Close'], errors='coerce')

# === Lag the Fama-French factors by 1 day ===
for col in ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']:
    df[col] = df.groupby('Company')[col].shift(1)

df = df.dropna(subset=['Price Close', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA'])

# === Create output folders ===
os.makedirs("optuna_results", exist_ok=True)
os.makedirs("optuna_models", exist_ok=True)

# === Sequence generator ===
def create_sequences(features, target, lookback):
    X, y = [], []
    for i in range(lookback, len(features)):
        X.append(features[i - lookback:i])
        y.append(target[i])
    return np.array(X), np.array(y)

# === Optimization function ===
def optimize_lstm(company_data, company_name):
    def objective(trial):
        lookback = trial.suggest_int('lookback', 1, 60, step=5)
        units = trial.suggest_int('units', 32, 256)
        dropout = trial.suggest_float('dropout', 0.1, 0.5)
        batch_size = trial.suggest_categorical('batch_size', [32, 64])
        epochs = trial.suggest_int('epochs', 20, 35)

        features = company_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']].values
        target = company_data['Price Close'].values.reshape(-1, 1)

        f_scaler = MinMaxScaler()
        t_scaler = MinMaxScaler()
        features_scaled = f_scaler.fit_transform(features)
        target_scaled = t_scaler.fit_transform(target)

        full_data = np.hstack((features_scaled, target_scaled))
        X, y = create_sequences(full_data, target_scaled, lookback)

        split = int(len(X) * 0.8)
        X_train, X_val = X[:split], X[split:]
        y_train, y_val = y[:split], y[split:]

        model = Sequential()
        model.add(LSTM(units, input_shape=(X.shape[1], X.shape[2])))
        model.add(Dropout(dropout))
        model.add(Dense(1))
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

        es = EarlyStopping(patience=5, restore_best_weights=True, verbose=0)
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                  validation_data=(X_val, y_val), callbacks=[es], verbose=0)

        y_pred = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))

        trial.set_user_attr("Company", company_name)
        return rmse

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=30, show_progress_bar=True)

    trial_df = study.trials_dataframe()
    trial_df['Company'] = company_name

    return study.best_trial.params, trial_df

# === Select chunk to run ===
chunk_number = 2  # Set this manually: 0, 1, 2, or 3
num_chunks = 4
all_companies = sorted(df['Company'].unique())
chunk_size = len(all_companies) // num_chunks
chunks = [all_companies[i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks - 1)]
chunks.append(all_companies[(num_chunks - 1)*chunk_size:])
companies_to_run = chunks[chunk_number]

# === Store all results ===
all_results = []
all_trials_df = []

# === Run optimization per company in this chunk ===
for company in tqdm(companies_to_run, desc=f"Optimizing chunk {chunk_number}"):
    company_data = df[df['Company'] == company].dropna()
    if len(company_data) < 100:
        continue

    print(f"\n🔍 Optimizing for {company}...")
    best_params, trial_df = optimize_lstm(company_data, company)

    result = {"Company": company}
    result.update(best_params)
    all_results.append(result)
    all_trials_df.append(trial_df)

    print(f"✅ Done: {company} — Best Params: {best_params}")

# === Save outputs for this chunk ===
pd.DataFrame(all_results).to_csv(f"optuna_best_params_chunk{chunk_number}.csv", index=False)
pd.concat(all_trials_df, ignore_index=True).to_csv(f"optuna_trials_chunk{chunk_number}.csv", index=False)

Optimizing chunk 2:   0%|          | 0/11 [00:00<?, ?it/s][I 2025-05-01 09:08:09,159] A new study created in memory with name: no-name-33d2701a-8508-422b-a6fd-d55b0b1efd2d



🔍 Optimizing for KLAC.O...


  0%|          | 0/30 [00:00<?, ?it/s]

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


Optimizing chunk 2:   0%|          | 0/11 [00:14<?, ?it/s]

...

[I 2025-05-01 10:57:35,400] Trial 27 finished with value: 0.04397613402942584 and parameters: {'lookback': 16, 'units': 144, 'dropout': 0.3375280893865391, 'batch_size': 32, 'epochs': 29}. Best is trial 25 with value: 0.037169379601971336.
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step


Optimizing chunk 2:  91%|█████████ | 10/11 [1:49:58<08:55, 535.28s/it]

[I 2025-05-01 10:58:07,192] Trial 28 finished with value: 0.04783585092988138 and parameters: {'lookback': 36, 'units': 105, 'dropout': 0.49682708204032433, 'batch_size': 32, 'epochs': 24}. Best is trial 25 with value: 0.037169379601971336.
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


Optimizing chunk 2: 100%|██████████| 11/11 [1:50:04<00:00, 600.40s/it]

[I 2025-05-01 10:58:13,473] Trial 29 finished with value: 0.055683613478702945 and parameters: {'lookback': 11, 'units': 65, 'dropout': 0.4446856048721665, 'batch_size': 64, 'epochs': 29}. Best is trial 25 with value: 0.037169379601971336.
✅ Done: NXPI.O — Best Params: {'lookback': 1, 'units': 131, 'dropout': 0.43439810513738364, 'batch_size': 32, 'epochs': 29}


In [ ]:
import pandas as pd
import numpy as np
import optuna
import os
import json
import warnings
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")

# === Load and prepare dataset ===
df = pd.read_csv("cleaned_stock_data_without_arm.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Company', 'Date'])

# Fix commas to save as numeric
df['Price Close'] = df['Price Close'].astype(str).str.replace(",", ".", regex=False)
df['Price Close'] = pd.to_numeric(df['Price Close'], errors='coerce')

# === Lag the Fama-French factors by 1 day ===
for col in ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']:
    df[col] = df.groupby('Company')[col].shift(1)

df = df.dropna(subset=['Price Close', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA'])

# === Create output folders ===
os.makedirs("optuna_results", exist_ok=True)
os.makedirs("optuna_models", exist_ok=True)

# === Sequence generator ===
def create_sequences(features, target, lookback):
    X, y = [], []
    for i in range(lookback, len(features)):
        X.append(features[i - lookback:i])
        y.append(target[i])
    return np.array(X), np.array(y)

# === Optimization function ===
def optimize_lstm(company_data, company_name):
    def objective(trial):
        lookback = trial.suggest_int('lookback', 1, 60, step=5)
        units = trial.suggest_int('units', 32, 256)
        dropout = trial.suggest_float('dropout', 0.1, 0.5)
        batch_size = trial.suggest_categorical('batch_size', [32, 64])
        epochs = trial.suggest_int('epochs', 20, 35)

        features = company_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']].values
        target = company_data['Price Close'].values.reshape(-1, 1)

        f_scaler = MinMaxScaler()
        t_scaler = MinMaxScaler()
        features_scaled = f_scaler.fit_transform(features)
        target_scaled = t_scaler.fit_transform(target)

        full_data = np.hstack((features_scaled, target_scaled))
        X, y = create_sequences(full_data, target_scaled, lookback)

        split = int(len(X) * 0.8)
        X_train, X_val = X[:split], X[split:]
        y_train, y_val = y[:split], y[split:]

        model = Sequential()
        model.add(LSTM(units, input_shape=(X.shape[1], X.shape[2])))
        model.add(Dropout(dropout))
        model.add(Dense(1))
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

        es = EarlyStopping(patience=5, restore_best_weights=True, verbose=0)
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                  validation_data=(X_val, y_val), callbacks=[es], verbose=0)

        y_pred = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))

        trial.set_user_attr("Company", company_name)
        return rmse

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=30, show_progress_bar=True)

    trial_df = study.trials_dataframe()
    trial_df['Company'] = company_name

    return study.best_trial.params, trial_df

# === Select chunk to run ===
chunk_number = 3  # Set this manually: 0, 1, 2, or 3
num_chunks = 4
all_companies = sorted(df['Company'].unique())
chunk_size = len(all_companies) // num_chunks
chunks = [all_companies[i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks - 1)]
chunks.append(all_companies[(num_chunks - 1)*chunk_size:])
companies_to_run = chunks[chunk_number]

# === Store all results ===
all_results = []
all_trials_df = []

# === Run optimization per company in this chunk ===
for company in tqdm(companies_to_run, desc=f"Optimizing chunk {chunk_number}"):
    company_data = df[df['Company'] == company].dropna()
    if len(company_data) < 100:
        continue

    print(f"\n🔍 Optimizing for {company}...")
    best_params, trial_df = optimize_lstm(company_data, company)

    result = {"Company": company}
    result.update(best_params)
    all_results.append(result)
    all_trials_df.append(trial_df)

    print(f"✅ Done: {company} — Best Params: {best_params}")

# === Save outputs for this chunk ===
pd.DataFrame(all_results).to_csv(f"optuna_best_params_chunk{chunk_number}.csv", index=False)
pd.concat(all_trials_df, ignore_index=True).to_csv(f"optuna_trials_chunk{chunk_number}.csv", index=False)

Optimizing chunk 3:   0%|          | 0/11 [00:00<?, ?it/s][I 2025-05-01 11:18:00,402] A new study created in memory with name: no-name-442cb343-ea41-47e7-9aca-5a4b851d5145



🔍 Optimizing for ON.O...


  0%|          | 0/30 [00:00<?, ?it/s]

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step


Optimizing chunk 3:   0%|          | 0/11 [00:45<?, ?it/s]

...

[I 2025-05-01 13:19:28,277] Trial 27 finished with value: 0.016697155748350494 and parameters: {'lookback': 1, 'units': 181, 'dropout': 0.36729182787639636, 'batch_size': 32, 'epochs': 32}. Best is trial 16 with value: 0.01660377733694018.
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


Optimizing chunk 3:  91%|█████████ | 10/11 [2:01:41<12:11, 731.44s/it]

[I 2025-05-01 13:19:42,119] Trial 28 finished with value: 0.03060065495789258 and parameters: {'lookback': 11, 'units': 143, 'dropout': 0.34952212829334856, 'batch_size': 32, 'epochs': 31}. Best is trial 16 with value: 0.01660377733694018.
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


Optimizing chunk 3: 100%|██████████| 11/11 [2:02:07<00:00, 666.09s/it]

[I 2025-05-01 13:20:07,415] Trial 29 finished with value: 0.028150644169272335 and parameters: {'lookback': 21, 'units': 172, 'dropout': 0.37464285306482775, 'batch_size': 32, 'epochs': 29}. Best is trial 16 with value: 0.01660377733694018.
✅ Done: ZS.O — Best Params: {'lookback': 1, 'units': 227, 'dropout': 0.1717298780534997, 'batch_size': 32, 'epochs': 32}
